# Notebook to tidy 1999 Polling Place Data

## Libraries

In [23]:
import pandas as pd
import xlrd
import os
from os import listdir
from os.path import isfile, join

## Functions

#### `get_files(filepath)`
* Takes a path to a folder
* Returns a list of files in that folder

In [24]:
# filter a list of files for xls files
def get_files(filepath):
    return [f for f in listdir(filepath) if isfile(join(filepath, f))]

In [25]:
filepath = '1999_referenda/polling_places'
print(get_files(filepath))

['ppsact.txt', 'ppsact.xls', 'ppsnsw.txt', 'ppsnsw.xls', 'ppsnt.txt', 'ppsnt.xls', 'ppsqld.txt', 'ppsqld.xls', 'ppssa.txt', 'ppssa.xls', 'ppstas.TXT', 'ppstas.xls', 'ppsvic.txt', 'ppsvic.xls', 'ppswa.txt', 'ppswa.xls']


#### `get_xls(files)`
* Takes a list of file names
* Returns those that are .xls files

In [26]:
# filter a list of files for xls files
def get_xls(files):
    r = []
    for file in files:
        if 'xls' in file:
            r.append(file)
    return r

In [27]:
filepath = '1999_referenda/polling_places'
files = get_files(filepath)
files_xls = get_xls(files)
print(files_xls)

['ppsact.xls', 'ppsnsw.xls', 'ppsnt.xls', 'ppsqld.xls', 'ppssa.xls', 'ppstas.xls', 'ppsvic.xls', 'ppswa.xls']


#### `get_polling_places(filename, columns)`
* Takes a path to a polling place file, and a list defining the columns we should end up with
* Returns a data frame containing the rows that represented polling places

#### A bit of an explanation

The files are a bit of a mess, they look like this:

![alt text](img/img1.png "Title")

The approach I take below is to loop through each row of the spreadsheet, and
* `if` the row doesn't have any blank cells in the first columns of the spreadsheet
* and `if` the row doesn't begin with polling place name
`then` append the row to a data frame

Then I use the file name to add a state column

This will result in code we can run across all the states spreadsheets to end up with one file

In [28]:
def get_polling_places(filename, columns):

    # get data
    x = xlrd.open_workbook(file)
    x_sheet = x.sheet_by_index(0)
    
    # data frame to preserve outside loop
    df_polling_places = pd.DataFrame()
    
    # for each row in the spreadsheet
    for rowindex in range(x_sheet.nrows):
        row_slice = x_sheet.row_slice(
            rowx=rowindex, # take the index of this loop, 0 -> max rows
            start_colx=0, # we care about the first 6 columns hence 0->6
            end_colx=6
        )
        
        # take keys off the row
        row = []
        for cell in row_slice:
            row.append(cell.value)

        # if there are any blank cells, this isn't a real row of data
        if row.count('') == 0:
            # except one of those rows is the header row, which we want to throw out as well
            if row[0] != 'Polling Place Name':  

                # make a data frame
                df = pd.DataFrame(data=[row], columns = columns)

                # if the data frame is empty, create it, if not, append to it
                if df_polling_places.empty:
                    df_polling_places = df
                else:
                    df_polling_places = df_polling_places.append(df)
    
    # work out state from filename
    # state will be the chars from 3 chars after the final slash to .
    # as the files are in the form 'ppsnt.xls'
    
    # everything after the final slash
    needle = file.rfind('/')
    length = len(file)
    filepart = file[needle+1:length]
    
    # from the third char to the dot
    needle = filepart.find('.')
    state = filepart[3:needle].upper()
    
    # create state column
    df_polling_places['state'] = state
    
    # put state column at front
    
    #Make a list of all of the columns in the df
    cols = list(df_polling_places.columns.values)
    
    # remove state from the list
    cols.pop(cols.index('state'))
    
    # put state at the front of this list
    df_polling_places = df_polling_places[['state']+cols]
    
    return df_polling_places

In [29]:
file = '1999_referenda/polling_places/ppsnt.xls'

columns = [
    'polling_place',
    'premises',
    'address',
    'suburb',
    'postcode',
    'wheelchair_access'
]

get_polling_places(file, columns).head(5)

,state,polling_place,premises,address,suburb,postcode,wheelchair_access
0,NT,Alice Springs,Civic Centre (Foyer),Todd St,ALICE SPRINGS,870.0,F
0,NT,Alyangula,Alyangula Area School,Flinders St,ALYANGULA,885.0,F
0,NT,Anula,Anula Primary School,Yanyula Dr,ANULA,812.0,A
0,NT,Bakewell,Bakewell Primary School,Hutchison Tce,PALMERSTON,830.0,F
0,NT,Batchelor,Batchelor Area School,Nurndina St,BATCHELOR,845.0,F


### Run over each file, merge data frame

In [30]:
filepath = '1999_referenda/polling_places/'

columns = [
    'polling_place',
    'premises',
    'address',
    'suburb',
    'postcode',
    'wheelchair_access'
]

# get xls files from filepath
files = get_files(filepath)
files_xls = get_xls(files)

df_polling_places = pd.DataFrame()

for file in files_xls:
    
    file = filepath + file
    df = get_polling_places(file, columns)

    # if the data frame is empty, create it, if not, append to it
    if df_polling_places.empty:
        df_polling_places = df
    else:
        df_polling_places = df_polling_places.append(df)

df_polling_places.head(3)

,state,polling_place,premises,address,suburb,postcode,wheelchair_access
0,ACT,Bonython,Bonython Primary School,Hurtle Ave,BONYTHON,2905.0,F
0,ACT,Calwell,Calwell High School,Casey Cres,CALWELL,2905.0,F
0,ACT,Canberra Hospital,The Canberra Hospital,Blding 2 Level 3 Yamba Dr,GARRAN,2605.0,F


### Dedup polling places
* Make sure we haven't ended up with any polling place duplicates

In [31]:
print('inital num rows: ' +  str(len(df_polling_places.index)))

df_polling_places = df_polling_places.drop_duplicates()

print('final num rows: ' +  str(len(df_polling_places.index)))

inital num rows: 7336
final num rows: 7335


### Save polling places to csv

In [32]:
outfile = '1999_referenda_output/polling_places.csv'

df_polling_places.to_csv(
    outfile,
    sep=',',
    index=False,
    encoding='UTF8'
)